# Settings

In [1]:
import sys
import os

os.chdir('..')

# Imports

In [2]:
! pip install editdistance

In [3]:
from scripts.model import device, CharBiLSTM
from scripts.data import create_data_loader, load_data
from scripts.preprocessing import get_typoglycemia_modified_data

from torch import nn, optim
import torch

from sklearn.model_selection import train_test_split

import editdistance

# Data

In [4]:
df = load_data(similarity_threshold=0.8)
df_typo = get_typoglycemia_modified_data(df)

## Splitting data into train, val, test

In [5]:
dev, test = train_test_split(df_typo, test_size=0.2)
train, validation = train_test_split(dev, test_size=0.2) 

## Getting dataloaders

In [6]:
train_loader = create_data_loader(train)
validation_loader = create_data_loader(validation)
test_loader = create_data_loader(test)

# Model 

In [7]:
input_size = 10   # Size of one-hot encoded input
hidden_size = 128  # Number of LSTM units
output_size = 10  # Size of output (number of characters)
num_layers = 1   # Number of LSTM layers

model = CharBiLSTM(input_size, hidden_size, output_size, num_layers).to(device)
loss_function = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training

In [8]:
for epoch in range(epochs):
    epoch_loss = 0.0    
    for idx, batch in enumerate(train_loader):
        feats_batch, label_batch = batch        

        optimizer.zero_grad()
        y = model.forward(feats_batch)

        loss = loss_function(y, label_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss_avg = epoch_loss/len(train_loader)
    


NameError: name 'epochs' is not defined

# Evaluation